In [1]:
%matplotlib inline
import sys
sys.path.append('../SoundSyn')
sys.path.append('../Thinkdsp')
import soundSyn
import thinkdsp

import numpy as np
import matplotlib.pyplot as plt

In [3]:
def overdrive(waveIn, threshold):  # 0 < threshold < 0.5
    waveOut = waveIn.copy()
    for i in range(len(waveOut.ys)):
        if (abs(waveIn.ys[i]) < threshold):
            waveOut.ys[i] = waveIn.ys[i] 
        else: 
            if (waveIn.ys[i] > 0):
                waveOut.ys[i] = threshold
            else:
                waveOut.ys[i] = -threshold
    waveOut.normalize()            
    return waveOut

def fuzz_conv(inlist, g):
    for i in range(len(inlist)):
        x = inlist[i]
        if (x == 0):
            inlist[i] = 0
        else:
            r = x / abs(x)
            inlist[i] = r * (1 - np.exp(g * r * x))  
    
def fuzz(waveIn, gain):
    waveOut = waveIn.copy()
    fuzz_conv(waveOut.ys, gain)
    waveOut.normalize()
    return waveOut

def wahwah(waveIn, damp, minf, maxf, fw):
    waveOut = waveIn.copy()
    fs = waveIn.framerate
    q1 = 2 * damp
    fc = minf
    f1 = 2 * np.sin((np.pi * fc)/fs)
    waveOut.ys[0] = f1 * waveIn.ys[0]
    yl = f1 * waveOut.ys[0]
    yaux = waveOut.ys[0]
    delta = fw/fs
    for i in range(1,len(waveIn.ys)):
        if ((fc + delta > maxf) or (fc + delta < minf)):
            delta = -delta
        fc = fc + delta
        f1 = 2 * np.sin((np.pi * fc)/fs)
        yh = waveIn.ys[i] - yl - q1 * yaux
        waveOut.ys[i] = f1 * yh + yaux
        yl = f1 * waveOut.ys[i] + yl
        yaux = waveOut.ys[i]
        
    maxy = max(abs(waveOut.ys))
    waveOut.ys = waveOut.ys / maxy
    return waveOut


def tremolo(waveIn, fm):
    waveOut = waveIn.copy()
    fs = waveIn.framerate
    alpha = 0.5
    for i in range(0, len(waveIn.ys)):
        waveOut.ys[i] = waveIn.ys[i] * (1 + alpha * np.sin(2 * np.pi * i * fm/fs))
    waveOut.normalize()
    return waveOut


def delay(waveIn, delay):
    waveOut = waveIn.copy()
    fs = waveIn.framerate
    amp = 0.7
    sampDelay = round(delay * fs)
    for i in range(0, sampDelay):
        waveOut.ys[i] = waveIn.ys[i]
    for i in range(sampDelay, len(waveIn.ys)):
        waveOut.ys[i] = (amp * waveIn.ys[i]) + (1-amp) * waveIn.ys[i - sampDelay]
    waveOut.normalize()
    return waveOut

In [22]:
ssET = soundSyn.SoundSyn()
#ssET.set_partial([1,0.8,0.7,0.9,0.5,0.7,0.5,0.5])  # guitar ??
ssET.set_file("bwvAnhII114.txt")
w = ssET.make_wave()
#w = overdrive(w, 0.2)
#w = tremolo(w, 10)
w = delay(w, 0.6)
w.make_audio()